In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('up_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/bareilly-to-agra: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF70CD39642+30946]
	(No symbol) [0x00007FF70CCEE3D9]
	(No symbol) [0x00007FF70CBE6FDA]
	(No symbol) [0x00007FF70CC3822C]
	(No symbol) [0x00007FF70CC3850C]
	(No symbol) [0x00007FF70CC7DCB7]
	(No symbol) [0x00007FF70CC5CAAF]
	(No symbol) [0x00007FF70CC7B041]
	(No symbol) [0x00007FF70CC5C813]
	(No symbol) [0x00007FF70CC2A6E5]
	(No symbol) [0x00007FF70CC2B021]
	GetHandleVerifier [0x00007FF70CE6F84D+1301229]
	GetHandleVerifier [0x00007FF70CE7BDC7+1351783]
	GetHandleVerifier [0x00007FF70CE72A13+1313971]
	GetHandleVerifier [0x00007FF70CD6DD16+245686]
	(No symbol) [0x00007FF70CCF759F]
	(No symbol) [0x00007FF70CCF3814]
	(No symbol) [0x00007FF70CCF39A2]
	(No symbol) [0x00007FF70CCEA3FF]
	BaseThreadInitThunk [0x00007FF8087C257D+29]
	RtlUserThreadStart [0x00007FF808ACAF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-ticke

NameError: name 'driver' is not defined

In [3]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0130,Janrath AC Seater 2+3,17:31,05h 00m,22:31,2.0,INR 520,48 Seats available
1,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - BLY0111,Janrath AC Seater 2+2,19:01,04h 44m,23:45,3.2,INR 598,36 Seats available
2,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - HDD0063,Ordinary Non AC Seater 2+3,19:39,07h 03m,02:42,3.3,INR 444,52 Seats available
3,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0128,Janrath AC Seater 2+2,20:01,05h 00m,01:01,4.1,INR 598,34 Seats available
4,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - STD0354,Ordinary Non AC Seater 2+3,21:27,06h 15m,03:42,3.3,INR 448,52 Seats available
...,...,...,...,...,...,...,...,...,...,...
790,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,International Tourist Centre,A/C Seater Push Back (2+2),22:30,07h 00m,05:30,0,INR 1199,37 Seats available
791,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,ROYAL SAFARI INDIA TRAVELS,Bharat Benz A/C Semi Sleeper (2+2),22:30,07h 30m,06:00,0,899,8 Seats available
792,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Jagdamba tourism,A/C Seater/Sleeper (2+1),22:30,07h 00m,05:30,0,INR 899,11 Seats available
793,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,KVS Holidays,A/C Seater / Sleeper (2+2),22:10,07h 15m,05:25,0,INR 1099,4 Seats available
